# Download data

You can compose queries to download raw data.

In [ ]:
import getpass
import platform

import matplotlib.pyplot as plt
import pandas as pd

import doplaydo.dodata as dd

username = getpass.getuser()
PROJECT_ID = f"spirals-{username}-{platform.node()}"

You have access to:

- dd.Project
- dd.Die
- dd.Wafer
- dd.ParentCell
- dd.Cell
- dd.Device
- dd.DeviceData

Where each model has its table columns as attributes.


![](https://i.imgur.com/CfOHl57.png)

You can use `get_data_by_query` to query a subset of data filtered by a list of clauses.

It will return a list of tuples where the first element is a `DeviceData` object and the second one is a pandas DataFrame.


In [ ]:
with dd.get_session() as session:
    data_tuples = dd.get_data_by_query(
        [
            dd.Project.project_id == PROJECT_ID,
        ],
        limit=1,
        session=session,
    )
    device_data, df = data_tuples[0]  # each tuple has DeviceData and pd.Dataframe
    device_id = device_data.device.device_id
device_data

In [ ]:
device_id

In [ ]:
df

In [ ]:
with dd.get_session() as session:
    data_tuples = dd.get_data_by_query(
        [
            dd.Project.project_id == PROJECT_ID,
        ],
        limit=1,
        session=session,
    )
    device_data, df = data_tuples[0]  # each tuple has DeviceData and pd.Dataframe
    device_id = device_data.device.device_id
    dfs = [dt[1] for dt in data_tuples]  # dataframes
    dds = [dt[0] for dt in data_tuples]  # device data objects
    plt.plot(df["wavelength"], df["output_power"])
    plt.xlabel("wavelength (nm)")
    plt.ylabel("output power (mW)")
    plt.title(device_id)

You can aggregate the dataframes and the device data objects

You can use the `DeviceData` object to traverse the data model and access additional fields.

You can go from DeviceData to any other tables by following the dashed arrows.

Each column is an attribute on the object representing the table.

![](https://i.imgur.com/CfOHl57.png)

In [ ]:
with dd.get_session() as session:
    data_tuples = dd.get_data_by_query(
        [
            dd.Project.project_id == PROJECT_ID,
        ],
        limit=1,
        session=session,
    )
    device_data, df = data_tuples[0]  # each tuple has DeviceData and pd.Dataframe
    device_id = device_data.device.device_id
    dfs = [dt[1] for dt in data_tuples]  # dataframes
    dds = [dt[0] for dt in data_tuples]  # device data objects
    print("device name: ", dds[0].device.device_id)
    print("die x: ", dds[0].die.x)
    print("die y: ", dds[0].die.y)
    print("wafer name: ", dds[0].die.wafer.wafer_id)
    print("cell name: ", dds[0].device.cell.cell_id)
    print("parent cell name: ", dds[0].device.parent_cell.cell_id)
    print("project name: ", dds[0].device.cell.project.project_id)
    # for example print the cell table and all its columns
    print(dds[0].device.cell)

## Build table for JMP

You can build a flat table by combining all the device data and pandas dataframes.  

In [ ]:
with dd.get_session() as session:
    data_tuples = dd.get_data_by_query(
        [
            dd.Project.project_id == PROJECT_ID,
            dd.Device.device_id
            == "RibLoss_cutback_rib_assembled_MFalse_W0p3_L0_20150_60150",
            dd.Die.x == 0,
            dd.Die.y == 0,
        ],
        session=session,
    )
    len(data_tuples)
    dds = [dt[0] for dt in data_tuples]  # device data objects
    dfs = [dt[1] for dt in data_tuples]  # dataframes
    dfs_all = []

    for device_data, df in zip(dds, dfs, strict=False):
        df["device_id"] = device_data.device.device_id
        df["die_x"] = device_data.die.x
        df["die_y"] = device_data.die.y
        df["wafer_id"] = device_data.die.wafer.wafer_id
        df["cell_id"] = device_data.device.cell.cell_id
        df["parent_cell_id"] = device_data.device.parent_cell.cell_id
        dfs_all.append(df)
dfs[0]

In [ ]:
dfs_all = pd.concat(dfs_all)  # You can concatenate all dataFrames together
dfs_all

## Advanced queries


To build advanced queries to filter metadata you can use the `attribute_filter` method. You can also use `or_`, `and_` for conditional clauses.

### Conditional filter

In [ ]:
with dd.get_session() as session:
    data_tuples = dd.get_data_by_query(
        [
            dd.Project.project_id == PROJECT_ID,
            dd.Device.device_id
            == "RibLoss_cutback_rib_assembled_MFalse_W0p3_L0_20150_60150",
        ],
        session=session,
    )
len(data_tuples)

By default anything you pass to the list joins the clauses as `and_`

In [ ]:
with dd.get_session() as session:
    data_tuples = dd.get_data_by_query(
        [
            dd.and_(
                dd.Project.project_id == PROJECT_ID,
                dd.Device.device_id
                == "RibLoss_cutback_rib_assembled_MFalse_W0p3_L0_20150_60150",
            )
        ],
        session=session,
    )
len(data_tuples)

You can also use an OR condition.

The query below filters by project name `PROJECT_ID` AND either  device  name (`rib_0p3` OR `rib_0p5`).

Because there are 21 measurements per device, one for each die, if you look for two specific devices, you will get 42 measurements.

In [ ]:
with dd.get_session() as session:
    data_tuples = dd.get_data_by_query(
        [
            dd.Project.project_id == PROJECT_ID,
            dd.or_(
                dd.Device.device_id
                == "RibLoss_cutback_rib_assembled_MFalse_W0p3_L0_20150_60150",
                dd.Device.device_id
                == "RibLoss_cutback_rib_assembled_MFalse_W0p5_L0_20250_492247",
            ),
        ],
        session=session,
    )
len(data_tuples)

You can also combine conditionals.

In the example below, you can get all the data for the specified project name, device name and either die coordinate.

In [ ]:
with dd.get_session() as session:
    data_tuples = dd.get_data_by_query(
        [
            dd.Project.project_id == PROJECT_ID,
            dd.Device.device_id
            == "RibLoss_cutback_rib_assembled_MFalse_W0p3_L0_20150_60150",
            dd.or_(
                dd.and_(dd.Die.x == 1, dd.Die.y == 1),  # either die 1,1
                dd.and_(dd.Die.x == 0, dd.Die.y == 0),  # or die 0,0
            ),
        ],
        session=session,
    )
len(data_tuples)

### Attribute filter

You can filter attributes of any data models by passing a list of conditions as shown below.

You can use `dd.Cell/dd.Wafer/...`, `key`, `value`

You can only filter for values that are str, bool, int, float.

The query below filters Cells with waveguide width 0.3um that are in a particular project and die coordinates.

In [ ]:
with dd.get_session() as session:
    device_data_objects = dd.get_data_by_query(
        [
            dd.attribute_filter(
                dd.Cell, "width_um", 0.3
            ),  # checking for values in JSON attributes
            dd.Project.project_id == PROJECT_ID,
            dd.Die.x == 0,
            dd.Die.y == 0,
        ],
        session=session,
    )

In [ ]:
len(device_data_objects)

You can combine conditional and attribute filter clauses together.

You can get the data for Cells for a given project with length_um 0um and either width_um 0.3um or 0.5um

In [ ]:
with dd.get_session() as session:
    data_tuples = dd.get_data_by_query(
        [
            dd.Project.project_id == PROJECT_ID,
            dd.attribute_filter(dd.Cell, "length_um", 0),
            dd.or_(
                dd.attribute_filter(dd.Cell, "width_um", 0.3),
                dd.attribute_filter(dd.Cell, "width_um", 0.5),
            ),
        ],
        session=session,
    )
len(data_tuples)